# 텍스트를 위한 딥러닝

## 자연어 처리 소개

## 텍스트 데이터 준비

### 텍스트 표준화

### 텍스트 분할 (토큰화)

### 어휘 인덱싱

어휘 사전 만들기

In [ ]:
vocabulary = {}
for text in dataset:
  text = standardize(text)
  tokens = tokenize(text)
  for token in tokens:
    if token not in vocabulary:
      vocabulary[token] = len(vocabulary)

벡터화

In [2]:
def one_hot_encode_token(token):
  vector = np.zeros((len(vocabulary),))
  token_index = vocabulary[token]
  vector[token_index] = 1
  return vector

#### Special tokens
- 0 : Masking token
  - 단어가 아니라 무시할 수 있음
  - 예를 들어, 시퀀스 데이터를 패딩할 때 사용됨
- 1 : OOV(Out of Vocabulary) token
  - 어휘 사전에 없는 단어
  - 예를 들어 `vocabulary["cherimoya"]`를 하면 `KeyError`가 발생
  - 이럴 때 `[UNK]` 같은 문자에 매칭시킴

### `TextVectorization` 층 사용하기

In [3]:
import string

class Vectorizer:
    def standardize(self, text):
        text = text.lower()
        return "".join(char for char in text if char not in string.punctuation)

    def tokenize(self, text):
        return text.split()

    def make_vocabulary(self, dataset):
        self.vocabulary = {"": 0, "[UNK]": 1}
        for text in dataset:
            text = self.standardize(text)
            tokens = self.tokenize(text)
            for token in tokens:
                if token not in self.vocabulary:
                    self.vocabulary[token] = len(self.vocabulary)
        self.inverse_vocabulary = dict(
            (v, k) for k, v in self.vocabulary.items())

    def encode(self, text):
        text = self.standardize(text)
        tokens = self.tokenize(text)
        return [self.vocabulary.get(token, 1) for token in tokens]

    def decode(self, int_sequence):
        return " ".join(
            self.inverse_vocabulary.get(i, "[UNK]") for i in int_sequence)

vectorizer = Vectorizer()
dataset = [
    "I write, erase, rewrite",
    "Erase again, and then",
    "A poppy blooms.",
]
vectorizer.make_vocabulary(dataset)

In [4]:
test_sentence = "I write, rewrite, and still rewrite again"
encoded_sentence = vectorizer.encode(test_sentence)
print(encoded_sentence)

[2, 3, 5, 7, 1, 5, 6]


In [5]:
decoded_sentence = vectorizer.decode(encoded_sentence)
print(decoded_sentence)

i write rewrite and [UNK] rewrite again


근데 이런 건 성능이 별로 안 좋아서 케라스 `TextVectorization` layer를 사용할거야

In [7]:
from tensorflow.keras.layers import TextVectorization
text_vectorization = TextVectorization(
    # output : 정수 인덱스로 인코딩된 단어 시퀀스
    output_mode="int",
)

**`TextVectorization` layer**  
- 보통 표준화를 위해 소문자 변환, 구두점 제거, 공백으로 토큰화
- 사용자 정의 함수 역시 사용 가능
  - `tf.string` 텐서를 이용해서 처리해야 함

In [8]:
import re
import string
import tensorflow as tf

def custom_standardization_fn(string_tensor):
    # 문자열을 소문자로 변환
    lowercase_string = tf.strings.lower(string_tensor)
    # 구두점 문자를 빈 문자로 변환
    return tf.strings.regex_replace(
        lowercase_string, f"[{re.escape(string.punctuation)}]", "")

def custom_split_fn(string_tensor):
    # 토큰화
    return tf.strings.split(string_tensor)

text_vectorization = TextVectorization(
    output_mode="int",
    standardize=custom_standardization_fn,
    split=custom_split_fn,
)

#### 어휘사전 인덱싱
- `adapt()` 메소드 사용

**Text corpus**  
- 텍스트 말뭉치
- 텍스트 처리 분야에서는 훈련 데이터를 말뭉치라고 부름

In [9]:
dataset = [
    "I write, erase, rewrite",
    "Erase again, and then",
    "A poppy blooms.",
]
text_vectorization.adapt(dataset)

**어휘 사전 출력하기**  
- 정수 시퀀스로 인코딩된 텍스트를 단어로 다시 변환할 때 유용
- 처음 두 항목 : masking token, OOV token
- 빈도 순으로 정렬

In [10]:
text_vectorization.get_vocabulary()

['',
 '[UNK]',
 'erase',
 'write',
 'then',
 'rewrite',
 'poppy',
 'i',
 'blooms',
 'and',
 'again',
 'a']

**예시 문장 디코딩**

In [11]:
vocabulary = text_vectorization.get_vocabulary()
test_sentence = "I write, rewrite, and still rewrite again"
encoded_sentence = text_vectorization(test_sentence)
print(encoded_sentence)

tf.Tensor([ 7  3  5  9  1  5 10], shape=(7,), dtype=int64)


In [ ]:
inverse_vocab = dict(enumerate(vocabulary))
decoded_sentence = " ".join(inverse_vocab[int(i)] for i in encoded_sentence)
print(decoded_sentence)

i write rewrite and [UNK] rewrite again


#### `tf.data` 파이프라인 또는 모델의 일부로 `TextVectorization` 층 사용하기
- TextVectorization 층은 대부분 dictionary lookup 연산이기 때문에 CPU에서만 실행
- 모델을 GPU로 훈련한다면, 이게 CPU에서 실행된 후 출력을 GPU로 보낼 것
- 이 과정에서 동기적으로 처리할 수도 있고 비동기적으로 처리할 수도 있음  
  

1. `tf.data` 파이프라인을 사용한 비동기적 처리
```python
int_sequence_dataset = string_dataset.map(
  text_vectorization,
  num_parallel_calls=4
)
```
  - `num_parallel_calls` : 여러 개의 CPU 코어에서 `map()` 메소드를 병렬화
  - GPU가 벡터화된 데이터 배치에서 모델을 실행할 때 CPU가 원시 문자열의 다음 배치를 벡터화
  - 텍스트 데이터를 비동기적으로 수행
  - GPU나 TPU를 사용해 최상의 성능을 얻기 위해 사용

2. 케라스 모델의 일부로 만드는 동기적 처리
```python
# 문자열을 받는 symbolic input
text_input = keras.Input(shape=(), dtype="string")
# Text vectorization
vectorized_text = text_vectorization(text_input)
# 일반적인 functional API model처럼 벡터화 층 추가
embedded_input = keras.layers.Embedding(...)(vectorized_text)
output = ...
model = keras.Model(text_input, output)
```
  - 모델의 나머지 부분과 동기적으로 수행됨
  - 훈련 단계마다 GPU에 놓인 모델의 나머지 부분이 실행되기 위해 CPU에 놓인 벡터화 층의 출력이 준비되기를 기다림
  - 모델을 제품 환경에 배포해야 한다면 해당 방법을 사용
    - 이 층이 포함되지 않으면 제품 환경에서 전처리를 다시 구현해야 함
    - 전처리에 작은 차이가 생기면 모델의 정확도에 손상 가능성이 있음
  - keras의 `TextVectorization` layer를 사용하면 모델에 텍스트 전처리를 포함시켜 쉽게 배포 가능
  - `tf.data` 파이프라인의 일부로 이 층을 사용하는 경우도 가능

## 단어 그룹을 표현하는 두 가지 방법: 집합과 시퀀스
- 단어는 범주형 특성 : 미리 정의된 집합에 있는 값
- 단어를 처리하기 위해서는 단어를 특성 공간의 차원으로 인코딩하거나 범주 벡터(단어 벡터)로 인코딩
- 단어 순서를 인코딩하는 방법 (단어를 문장으로 구성하는 방식)
  - 문장에 있는 단어는 자연스럽고 표준이 되는 순서가 없음
  - 어떻게 단어의 순서를 표현하는지가 NLP 아키텍처의 핵심 질문
  - BoW 모델 : 단어의 순서를 무시, 텍스트를 순서가 없는 단어의 집합으로 처리
  - 시퀀스 모델 : 한 번에 하나의 단어씩 등장하는 순서대로 처리
  - 하이브리드 모델 - 트랜스포머 아키텍쳐
    - 순서에 구애받지 않지만 처리하는 표현에 단어 위치 정보를 주입
    - 순서를 고려하면서 문장의 여러 부분을 동시에 보기 가능

### IMDB 영화 리뷰 데이터 준비하기

In [20]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  4241k      0  0:00:19  0:00:19 --:--:-- 8378k


In [15]:
!ls -al aclImdb/train

total 66632
drwxr-xr-x 5 7297 1000     4096 Feb  4 03:27 .
drwxr-xr-x 4 7297 1000     4096 Jun 26  2011 ..
-rw-r--r-- 1 7297 1000 21021197 Apr 12  2011 labeledBow.feat
drwxr-xr-x 2 7297 1000   385024 Feb  4 03:27 neg
drwxr-xr-x 2 7297 1000   372736 Feb  4 03:27 pos
drwxr-xr-x 2 7297 1000  1409024 Feb  4 03:27 unsup
-rw-r--r-- 1 7297 1000 41348699 Apr 12  2011 unsupBow.feat
-rw-r--r-- 1 7297 1000   612500 Apr 12  2011 urls_neg.txt
-rw-r--r-- 1 7297 1000   612500 Apr 12  2011 urls_pos.txt
-rw-r--r-- 1 7297 1000  2450000 Apr 12  2011 urls_unsup.txt


neg : 부정적인 리뷰
pos : 긍정적인 리뷰

In [21]:
# 지금은 필요없어서 삭제
!rm -r aclImdb/train/unsup

In [17]:
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

검증 세트 만들기

In [ ]:
import os, pathlib, shutil, random

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    # 코드를 여러 번 실행해도 동일한 검증 세트가 만들어지도록
    # 랜덤 시드를 지정하여 훈련 파일 목록을 섞음
    random.Random(1337).shuffle(files)
    # 훈련 파일 중 20%를 검증 파일로 사용
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    # 파일을 aclImdb/val/neg 와 aclImdb/val/pos 로 옮김
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

훈련, 검증, 테스트를 위한 객체 만들기

In [24]:
from tensorflow import keras
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

Found 25000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


**첫 번째 배치의 크기와 dtype 출력하기**

In [25]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b"Actually, the movie is neither horror nor Sci-Fi. With a very strong Christian religious theme, this movie delivers minimal content and no suspense. Second-tier actors do half-decent jobs of reading their boring roles. The only good performance is by Sydney Penny who plays a role of a mother of ... I won't spoil the movie, it's either Christ or Anti-Christ. Avoid watching this movie unless you a Christian religious fanatic obsessed with apocalypse.<br /><br />Being a non-Christian, I had to force myself to watch this movie just because I wanted to write this review. It's a pity that Sci-Fi channel had to air this movie at the peak evening time.", shape=(), dtype=string)
targets[0]: tf.Tensor(0, shape=(), dtype=int32)


### 단어를 집합으로 처리하기: BoW 방식
- 순서를 무시하고 토큰의 집합으로 다루기
- 개별 단어(unigram)를 사용
- 혹은 연속된 토큰 그룹(N-그램)으로 국부적인 순서 정보 유지

#### Single words (unigrams) with binary encoding
"the cat sate on the mat" -> `{"cat", "mat", "on", "sat", "the"}`
- 전체 텍스트를 하나의 벡터로 표현 가능한 장점
- 벡터의 각 원소는 한 단어의 존재 유무를 표시
- 예를 들어 multi-hot binary encoding을 사용하면 하나의 텍스트를 어휘 사전에 있는 단어 개수만큼의 차원을 가진 벡터로 인코딩
- 텍스트에 있는 단어에 해당하는 차원은 1이고 나머지는 0

**`TextVectorization` 층으로 데이터 전처리하기**

In [26]:
text_vectorization = TextVectorization(
    # 가장 많이 등장하는 2만개로 어휘 사전 제한
    # 한 두 번만 등장하는 단어들은 유용하지도 않고 학습 시 편견을 가지기 쉬움
    max_tokens=20000,
    # 멀티-핫 이진 벡터로 출력 토큰을 인코딩
    output_mode="multi_hot",
)

# 레이블 없이 원시 텍스트 입력만 반환하는 데이터셋 준비
text_only_train_ds = train_ds.map(lambda x, y: x)
# adapt() 메소드로 데이터셋의 어휘 사전을 인덱싱
text_vectorization.adapt(text_only_train_ds)

# 전처리, multi core 활용을 위해 num_parallel_calls 지정
binary_1gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_1gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_1gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

**이진 유니그램 데이터셋의 출력 확인하기**

In [27]:
for inputs, targets in binary_1gram_train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32, 20000)
inputs.dtype: <dtype: 'int64'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor([1 1 1 ... 0 1 0], shape=(20000,), dtype=int64)
targets[0]: tf.Tensor(1, shape=(), dtype=int32)


**모델 생성 함수**

In [28]:
from tensorflow import keras
from tensorflow.keras import layers

def get_model(max_tokens=20000, hidden_dim=16):
    inputs = keras.Input(shape=(max_tokens,))
    x = layers.Dense(hidden_dim, activation="relu")(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop",
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
    return model

**이진 유니그램 모델 훈련하고 테스트하기**

In [29]:
model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_1gram.h5",
                                    save_best_only=True)
]

# 데이터셋의 cache() 메소드를 호출하여 메모리에 캐싱
# --> 첫 번째 에포크에서 한 번만 전처리하고 이후 에포크에서는 전처리된 텍스트를 재사용
#     메모리에 들어갈 수 있는 작은 데이터일 때 사용 가능
model.fit(binary_1gram_train_ds.cache(),
          validation_data=binary_1gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)
model = keras.models.load_model("binary_1gram.h5")
print(f"테스트 정확도: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 20000)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 16)                  │         320,016 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 320,033 (1.22 MB)

 Trainable params: 320,033 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

...

782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9259 - loss: 0.2237 - val_accuracy: 0.9554 - val_loss: 0.1514
Epoch 8/10
781/782 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9294 - loss: 0.2235

782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9294 - loss: 0.2235 - val_accuracy: 0.9580 - val_loss: 0.1453
Epoch 9/10
771/782 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9313 - loss: 0.2215

782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9313 - loss: 0.2215 - val_accuracy: 0.9568 - val_loss: 0.1450
Epoch 10/10
778/782 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9325 - loss: 0.2090

782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9325 - loss: 0.2090 - val_accuracy: 0.9556 - val_loss: 0.1424


782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8832 - loss: 0.3572
테스트 정확도: 0.883


- 긍정/부정 두 가지의 균형 잡힌 이진 분류 데이터셋이기 때문에 단순한 기준점은 **50%**
- 해당 데이터셋의 최상의 테스트 정확도는 약 95%

#### 이진 인코딩을 사용한 바이그램
- 보통 하나의 개념이 여러 단어로 표현 가능하기 때문에 단어 순서를 무시하는 것은 매우 파괴적임
- 예를 들어, "United States"는 "states"와 "united" 단어의 개별적 의미와 많이 다른 개념을 제공
- 따라서 단일 단어가 아닌 N-그램을 사용하여 국부적인 순서 정보를 BoW 표현에 추가해야 함
"the cat sate on the mat"
  - `{"the", "the cat", "cat", "cat sat", "sat", "sat on", "on", "on the", "the mat"}`


**바이그램을 반환하는 `TextVectorization` 층 만들기**

In [30]:
text_vectorization = TextVectorization(
    # 매개변수를 전달하여 bigram, trigram 등 설정
    ngrams=2,
    max_tokens=20000,
    output_mode="multi_hot",
)

**이진 바이그램 모델 훈련하고 테스트하기**

In [31]:
text_vectorization.adapt(text_only_train_ds)
binary_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_2gram.h5",
                                    save_best_only=True)
]
model.fit(binary_2gram_train_ds.cache(),
          validation_data=binary_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)
model = keras.models.load_model("binary_2gram.h5")
print(f"테스트 정확도: {model.evaluate(binary_2gram_test_ds)[1]:.3f}")

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 20000)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 16)                  │         320,016 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 320,033 (1.22 MB)

 Trainable params: 320,033 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

...

782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9490 - loss: 0.1789 - val_accuracy: 0.9694 - val_loss: 0.1058
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9508 - loss: 0.1681

782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9508 - loss: 0.1681 - val_accuracy: 0.9724 - val_loss: 0.1019
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9509 - loss: 0.1740 - val_accuracy: 0.9746 - val_loss: 0.1025
Epoch 9/10
780/782 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9535 - loss: 0.1694

782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9535 - loss: 0.1694 - val_accuracy: 0.9762 - val_loss: 0.0998
Epoch 10/10
773/782 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9526 - loss: 0.1772

782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9526 - loss: 0.1773 - val_accuracy: 0.9756 - val_loss: 0.0991


782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8955 - loss: 0.3441
테스트 정확도: 0.893


테스트 성능이 향상됨! --> 국부적인 순서 아주 중요하다±!

#### TF-IDF 인코딩을 사용한 바이그램
- 개별 단어나 N-그램의 등장 횟수를 카운트한 정보를 추가
- 즉, 텍스트에 대한 단어의 히스토그램을 사용!
  - `{"the": 2, "the cat": 1, "cat": 1, "cat sat": 1, "sat": 1, "sat on": 1, "on": 1, "on the": 1, "the mat": 1, "mat": 1}`
- 텍스트 분류 작업 : 한 샘플에 단어가 얼마나 많이 등장하는지가 중요
  - 예를 들어, 긴 영화 리뷰라면 "terrible"이란 단어 포함 가능성 있음 근데 많이 포함되어 있음 당연 부정적이겠지?

**토큰 카운트를 반환하는 `TextVectorization` 층**

In [32]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="count"
)

근데 "the", "a", "is", "are" 이런거는 부정/긍정에 연관없지만 많이 등장함
- 정규화 사용하여 단어 카운트 정규화
- 근데 벡터화된 문장 대부분은 거의 전체가 0임
- Sparsity(희소성)가 있음
  - 계산 부하를 줄임
  - 과대적합 위험을 감소
- 그럼 어떻게 정규화하지? -> 나눗셈만 이용

**TF-IDF 정규화 (Term Frequency - Inverse Document Frequency)**  
- 데이터셋에 있는 모든 문서에 걸쳐 단어가 등장한다면 별로 유용하지 않음
- 전체 텍스트 데이터셋 중 일부에서만 나타나는 단어는 매우 독특하므로 중요함
- 따라서, 이렇게 계산해보자!!
  - '단어 빈도'로 해당 단어에 가중치를 부여 : 현재 문서에 단어가 등장하는 횟수
  - '문서 빈도'로 나누기 : 데이터셋 전체에 단어가 등장하는 횟수
```python
def tfidf(term, document, dataset):
  trum_freq = document.count(term)
  doc_freq = math.log(sum(doc.count(term) for doc in dataset) + 1)
  return term_freq / doc_freq
```

**TF-IDF 가중치가 적용된 출력을 반환하는 `TextVectorization` 층**

In [33]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    # 이미 구현되어 있기 때문에 매개변수로 tf_idf를 설정하면 됨
    output_mode="tf_idf",
)

**TF-IDF 바이그램 모델 훈련하고 테스트하기**

In [34]:
# 텐서플로 2.8.x 버전에서 TF-IDF 인코딩을 GPU에서 수행할 때 오류가 발생할 수 있습니다.
# 텐서플로 2.9에서 이 이슈가 해결되었지만 코드를 테스트할 시점에 코랩의 텐서플로 버전은 2.8.2이기 때문에
# 에러를 피하기 위해 CPU를 사용하여 텍스트를 변환합니다.
# with tf.device("cpu"):
#     text_vectorization.adapt(text_only_train_ds)
# adapt() 메소드를 호출하면 어휘 사전과 TF-IDF 가중치 학습 가능
text_vectorization.adapt(text_only_train_ds)

tfidf_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
tfidf_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
tfidf_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("tfidf_2gram.h5",
                                    save_best_only=True)
]
model.fit(tfidf_2gram_train_ds.cache(),
          validation_data=tfidf_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)
model = keras.models.load_model("tfidf_2gram.h5")
print(f"테스트 정확도: {model.evaluate(tfidf_2gram_test_ds)[1]:.3f}")

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 20000)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 16)                  │         320,016 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 320,033 (1.22 MB)

 Trainable params: 320,033 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

...

782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9020 - loss: 0.2456 - val_accuracy: 0.9502 - val_loss: 0.1489
Epoch 7/10
769/782 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9075 - loss: 0.2357

782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9075 - loss: 0.2356 - val_accuracy: 0.9482 - val_loss: 0.1388
Epoch 8/10
775/782 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9136 - loss: 0.2256

782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9135 - loss: 0.2257 - val_accuracy: 0.9556 - val_loss: 0.1353
Epoch 9/10
777/782 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9179 - loss: 0.2135

782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9179 - loss: 0.2135 - val_accuracy: 0.9574 - val_loss: 0.1268
Epoch 10/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9213 - loss: 0.2082 - val_accuracy: 0.9626 - val_loss: 0.1318


782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8903 - loss: 0.3160
테스트 정확도: 0.891


89.8% -> 딱히 도움이 되는 것 같지는 않음  
근데 보통은 1퍼센트정도 성능이 올라가긴 함

**원시 문자열을 처리하는 모델 내보내기**
- tf.data 파이프라인의 일부로 텍스트 표준화, 분할, 인덱싱을 수행함
- 이 파이프라인과 독립적으로 실행되는 모델을 추출해야 한다면 자체적인 텍스트 전처리를 사용해야 함
  - 그렇지 않으면 제품 환경에서 다시 구현해야 하는데 이게 성능 차이의 원인이 될 수도 있음


In [35]:
# 하나의 입력은 하나의 문자열
inputs = keras.Input(shape=(1,), dtype="string")
# 텍스트 전처리
processed_inputs = text_vectorization(inputs)
# 훈련된 모델을 적용
outputs = model(processed_inputs)
# end to end model
inference_model = keras.Model(inputs, outputs)

만들어진 모델을 사용해서 raw 문자열의 배치를 처리

In [36]:
import tensorflow as tf

raw_text_data = tf.convert_to_tensor([
    ["That was an excellent movie, I loved it."],
])
predictions = inference_model(raw_text_data)
print(f"긍정적인 리뷰일 확률: {float(predictions[0] * 100):.2f} 퍼센트")

긍정적인 리뷰일 확률: 98.70 퍼센트
